### This notebook consist of code for creating the html files for the website each time data is updated.

##### Set-up

In [32]:
import json
import os
from pathlib import Path
import glob
import shutil
import pandas as pd
from jinja2 import Environment, FileSystemLoader
from jinja2.exceptions import UndefinedError

In [33]:
def delete_ds_store(path):

    ds_store_files = glob.glob(os.path.join(path, '**', '.DS_Store'), recursive=True)
    

    for file_path in ds_store_files:
        os.remove(file_path)
        print(f"Deleted: {file_path}")
        

current_directory = os.getcwd()


delete_ds_store(current_directory)


DEFINING CONSTANTS

In [74]:
# Defining Paths
GROUP_DATA_DIR = Path("../../group-data")
TEMPLATE_DIR_PATH = GROUP_DATA_DIR.parent / "dtiwebsite_generator" / "templates"
WEBSITE_DATA_PATH = GROUP_DATA_DIR / "website_data/"
HOSTING_PATH = GROUP_DATA_DIR.parent / "deepthought-initiative.github.io"
CONTENT_DIR_PATH = WEBSITE_DATA_PATH / "content"
MEMBERS_DIR = GROUP_DATA_DIR / "members"

JSON_FILES_TO_LOAD = [
    "general",
    "homepage",
    "research",
    "support",
    "contact",
]

INSTITUTION_FILTER = "Michigan State University"
GROUP_FILTER = ["DTI", "TARDIS", "ICER", "kerzendorf"]

INDIVIDUAL_MEMBER_SECTION_MAP = {
    "education": "Education",
    "experiences": "Experience",
    "projects": "Projects",
    "awards": "Awards & Recognition",
    "outreach": "Outreach Programs",
}

In [34]:
# # Copying people directories

# def copy_folders(source, destination):
#     try:
#         shutil.copytree(source, destination)
#         print("Folders copied successfully!")
#     except shutil.Error as e:
#         print(f"Error copying folders: {e}")
#     except OSError as e:
#         print(f"Error creating destination directory: {e}")

# source_dir = "../temp/group-data/members"
# destination_dir = "../group-data/members"

# for folder_name in os.listdir(source_dir):
#     folder_path = os.path.join(source_dir, folder_name)
#     if os.path.isdir(folder_path):
#         json_path = os.path.join(folder_path, "jsons", "basic_info.json")
#         if os.path.exists(json_path):
#             with open(json_path) as f:
#                 data = json.load(f)
#                 display_info = data.get("display", {})
#                 if display_info.get("dti", True):
#                     destination_folder = os.path.join(destination_dir, folder_name)
#                     copy_folders(folder_path, destination_folder)

In [35]:
# # Copying common article content
# source_folder = '../temp/group-data/website_data'
# destination_folder = '../group-data/website_data'

# def copy_json_files(source_folder, destination_folder):
#     source_content_folder = os.path.join(source_folder, 'content')
#     destination_content_folder = os.path.join(destination_folder, 'content')
#     destination_website_files_folder = os.path.join(destination_folder, 'website_files')

#     os.makedirs(destination_content_folder, exist_ok=True)
#     os.makedirs(destination_website_files_folder, exist_ok=True)

#     for filename in os.listdir(source_content_folder):
#         if filename.endswith('.json'):
#             file_path = os.path.join(source_content_folder, filename)
#             with open(file_path, 'r') as file:
#                 try:
#                     json_data = json.load(file)
#                     platforms = json_data.get('platforms')
#                     if platforms and 'dti' in platforms:
#                         destination_path = os.path.join(destination_content_folder, filename)
#                         shutil.copy(file_path, destination_path)
#                         print(f"Copied file: {filename} from '{source_content_folder}' to '{destination_content_folder}'")

#                         process_cover_image(json_data, source_folder, destination_website_files_folder)
#                         process_content_images(json_data, source_folder, destination_website_files_folder)

#                 except json.JSONDecodeError:
#                     print(f"Error decoding JSON in file: {file_path}")

# def process_cover_image(json_data, source_folder, destination_website_files_folder):
#     cover_image_path = json_data.get('cover_image')
#     if cover_image_path:
#         source_image_path = os.path.join(source_folder, cover_image_path)
#         if os.path.exists(source_image_path):
#             destination_image_path = os.path.join(destination_website_files_folder, 'images', 'article_content', os.path.basename(cover_image_path))
#             os.makedirs(os.path.dirname(destination_image_path), exist_ok=True)
#             shutil.copy(source_image_path, destination_image_path)
#             print(f"Copied file: {cover_image_path} from '{source_folder}' to '{destination_image_path}'")

# def process_content_images(json_data, source_folder, destination_website_files_folder):
#     content = json_data.get('content')
#     if content:
#         for key, value in content.items():
#             if key.startswith('img'):
#                 image_path = value
#                 if image_path:
#                     source_image_path = os.path.join(source_folder, image_path)
#                     if os.path.exists(source_image_path):
#                         destination_image_path = os.path.join(destination_website_files_folder, 'images', 'article_content', os.path.basename(image_path))
#                         os.makedirs(os.path.dirname(destination_image_path), exist_ok=True)
#                         shutil.copy(source_image_path, destination_image_path)
#                         print(f"Copied file: {image_path} from '{source_folder}' to '{destination_image_path}'")

# copy_json_files(source_folder, destination_folder)


In [36]:
def page_link(a):
    """Return the HTML file name after replacing blank spaces(" ") with underscores("-")"""
    return a.replace(" ", "_") if " " in a else a

In [37]:
environment = Environment(
    loader=FileSystemLoader(TEMPLATE_DIR_PATH),
    extensions=["jinja2.ext.loopcontrols", "jinja2.ext.do"],
)
environment.globals["page_link"] = page_link

In [39]:
def create_page(template, html, **kwargs):
    """
    Create an HTML page using a Jinja2 template and save it to a specified path.

    Parameters:
    ----------
    template : str
        The filename of the Jinja2 template to be used.
    html : str
        The filename of the HTML file to be generated.
    **kwargs : dict
        Additional keyword arguments to be passed to the Jinja2 template for rendering.

    Returns:
    -------
    None

    """
    page_template = environment.get_template(template)
    template_level = html.count("/")
    page_html_path = HOSTING_PATH / html
    page_content = page_template.render(TEMPLATE_LEVEL=template_level, **kwargs)
    with open(page_html_path, mode="w", encoding="utf-8") as page:
        page.write(page_content)

In [40]:
def loading_general_json_files(file_to_load):
    """
    Load data from JSON files specified in a list of file names.

    Parameters:
    ----------
    json_data_list : list of str
        A list of file names (without extension) to load as JSON.

    Returns:
    -------
    dict
        A dictionary where keys are file names and values are the corresponding JSON data.

    Raises:
    ------
    FileNotFoundError:
        If a specified file does not exist.
    json.JSONDecodeError:
        If there's an issue decoding the JSON content from a file.

    """
    loaded_data = {}
    file_matches = WEBSITE_DATA_PATH/ f"{file_to_load}.json"

    if file_matches:
        try:
            with open(file_matches, "r") as json_file:
                loaded_data = json.load(json_file)
        except json.JSONDecodeError:
            print(f"Error decoding JSON in '{file_matches}'.")
    else:
        print(f"File '{file_to_load}.json' not found.")

    return loaded_data

In [41]:
article_content_list = []

for content_file_name in CONTENT_DIR_PATH.iterdir():
    with open(content_file_name, "r") as fcontent:
        article_content = json.load(fcontent)
    article_content_list.append(article_content)
article_content_df = pd.DataFrame(article_content_list)

# Sort the entire DataFrame by "category" and "date" in descending order
sorted_article_data = article_content_df.sort_values(
    by=["category", "date"], ascending=[True, False]
)
# Get the first row for each category using groupby and head
recent_content = sorted_article_data.groupby("category").head(1).copy()
recent_content["date"] = pd.to_datetime(
    recent_content["date"], format="%m-%d-%Y"
)
recent_content = recent_content.sort_values(by="date", ascending=False)

In [65]:
news_content_df = article_content_df[
    (article_content_df["category"] == "News")
    | (
        article_content_df["tags"].apply(
            lambda x: "news" in x if isinstance(x, list) else False
        )
    )
].sort_values(by=["date"], ascending=[False])

In [66]:
research_content_df = article_content_df[
    article_content_df["category"] != "News"
].sort_values(by=["category", "date"], ascending=[True, False])

In [68]:
# Reading info.json
info_json_list = []
for single_info_file_path in MEMBERS_DIR.glob("*/info.json"):
    with open(single_info_file_path, 'r') as f_info:
        member_data = json.load(f_info)
    info_json_list.append(member_data)
info_json_df = pd.DataFrame(info_json_list)
info_json_df.set_index("id", inplace=True)
info_json_df["full_name"] = info_json_df.apply(
    lambda row: (
        row["nick_name"] + " " + row["last_name"]
        if pd.notna(row["nick_name"])
        else row["first_name"] + " " + row["last_name"]
    ),
    axis=1,
)

In [45]:
general = loading_general_json_files("general")
homepage = loading_general_json_files("homepage")
contact = loading_general_json_files("contact")
research = loading_general_json_files("research")
support = loading_general_json_files("support")
create_page(
    "homepage.html.j2",
    "index.html",
    general=general,
    homepage=homepage,
    recent_content=recent_content.to_dict(orient="records"),
)
create_page("support.html.j2", "Support.html", general=general, support=support)
create_page(
    "news.html.j2",
    "News.html",
    general=general,
    content=news_content_df,
    category="news",
    member_data=info_json_df.to_dict("index"),
)
create_page(
    "research.html.j2",
    "Research.html",
    general=general,
    content=research_content_df,
    current_members=info_json_df.to_dict("index"),
    research=research,
)
create_page("contact.html.j2", "Contact.html", general=general, contact=contact)

In [69]:
for ind_news_keys, ind_news_values in news_content_df.iterrows():
    folder_path = (
        HOSTING_PATH / "news" / page_link(ind_news_values.article_id.lower())
    )
    create_page(
        "news_page_no_twitter.html.j2",
        f"news/{page_link(ind_news_values.article_id.lower())}.html",
        general=general,
        content=ind_news_values,
        member_data=info_json_df.to_dict("index"),
        category="News",
    )

In [70]:
folder_path = HOSTING_PATH / "sub_research"
folder_path.mkdir(parents=True, exist_ok=True)

for category in article_content_df.loc[article_content_df.category != "News", "category"].unique():
    create_page(
        "sub_research_frontpage.html.j2",
        f"sub_research/{page_link(category.lower())}.html",
        general=general,
        research=research,
        content=research_content_df,
        category=category,
        current_members=info_json_df.to_dict("index"),
    )

In [48]:
for ind_research_keys, ind_research_values in research_content_df.iterrows():
    folder_path = (
        HOSTING_PATH
        / "sub_research"
        / page_link(ind_research_values.category.lower())
    )
    folder_path.mkdir(parents=True, exist_ok=True)
    create_page(
        "research_page_no_twitter.html.j2",
        f"sub_research/{page_link(ind_research_values.category.lower())}/{page_link(ind_research_values.article_id.lower())}.html",
        general=general,
        content=ind_research_values,
        member_data=info_json_df.to_dict("index"),
        article_id=ind_research_values["article_id"],
    )

In [49]:
# Function for reading member data files
def read_member_data_jsons(file_to_read):
    data_list = []
    for single_member_file_path in MEMBERS_DIR.rglob(f"{file_to_read}"):
        with open(single_member_file_path, "r") as fname:
            member_data = json.load(fname)
        info_json_file_path = single_member_file_path.parent.parent / "info.json"
        with open(info_json_file_path, "r") as file_info:
            member_info_data = json.load(file_info)
        mem_id = member_info_data["id"]
        for single_entry in member_data:
            single_entry["id"] = mem_id
            data_list.append(single_entry)
    member_data_df = pd.DataFrame(data_list)
    member_data_df.set_index("id", inplace=True)
    return member_data_df

In [71]:
exp_df = read_member_data_jsons("experiences.json")[
    [
        "role",
        "start_date",
        "end_date",
        "institution",
        "group",
    ]
]
edu_df = read_member_data_jsons("education.json")[
    ["start_date", "end_date", "institution", "subject", "degree"]
]

exp_df_grouped = (
    exp_df.groupby("id")
    .apply(lambda x: x.to_dict(orient="records"))
    .reset_index(name="info")
    .set_index("id")
)
edu_df_grouped = (
    edu_df.groupby("id")
    .apply(lambda x: x.to_dict(orient="records"))
    .reset_index(name="info")
    .set_index("id")
)

In [72]:
social_link_list = []
for single_member_file_path in MEMBERS_DIR.rglob("social_links.json"):
    with open(single_member_file_path, "r") as fname:
        member_social_link = json.load(fname)
    info_json_file_path = single_member_file_path.parent.parent / "info.json"
    with open(info_json_file_path, "r") as file_info:
        member_info_data = json.load(file_info)
    mem_id = member_info_data["id"]
    member_social_link["id"] = mem_id
    social_link_list.append(member_social_link)
social_links_df = pd.DataFrame(social_link_list)
social_links_df.set_index("id", inplace=True)
social_links_df.fillna("", inplace=True)

In [52]:
filtered_exp_df = exp_df[exp_df["end_date"].isna()]
def most_recent_row(group):
    return group[group["start_date"] == group["start_date"].max()]
filtered_exp_df_most_recent = (
    filtered_exp_df.groupby("id").apply(most_recent_row).droplevel(0)
)

In [53]:
filtered_edu_df = edu_df[edu_df["end_date"].isna()]

In [54]:
exp_ids = filtered_exp_df_most_recent.index.unique()
edu_ids = filtered_edu_df.index.unique()
common_ids = list(set(exp_ids).union(edu_ids))
current_member_df = info_json_df.loc[common_ids]

In [55]:
for m_key, m_value in current_member_df.iterrows():
    if m_key in filtered_edu_df.index:
        if (
            filtered_edu_df.loc[m_key, "degree"] == "Bachelors"
            and filtered_edu_df.loc[m_key, "institution"] == INSTITUTION_FILTER
        ):
            current_academic_role = "Undergraduate Student"
        if filtered_edu_df.loc[m_key, "degree"] in ["PhD", "Masters"]:
            current_academic_role = "Graduate Student"
    elif m_key in filtered_exp_df_most_recent.index:
        current_academic_role = filtered_exp_df_most_recent.loc[m_key, "role"]
    else:
        current_academic_role = ""
    current_member_df.loc[m_key,"academic_role"] = current_academic_role

In [56]:
projects_df = read_member_data_jsons("projects.json")
sorted_projects_df = projects_df.sort_values(by=["end_date"], ascending=False)
awards_df = read_member_data_jsons("awards.json")

for mem_key, mem_value in current_member_df.iterrows():
    mem_projects = sorted_projects_df.loc[mem_key]
    if not mem_projects.empty:
        if isinstance(mem_projects, pd.Series):
            current_project_title = mem_projects["project_title"]
        else:
            current_project_title = mem_projects.iloc[0]["project_title"]
    else:
        current_project_title = ""
    current_member_df.loc[mem_key, "current_project_title"] = current_project_title

In [73]:
create_page(
    "current_members.html.j2",
    "current_members.html",
    general=general,
    current_members=current_member_df.to_dict("index"),
    socials=social_links_df.to_dict("index"),
)

In [58]:
def most_recent_row(group):
    return group[group["end_date"] == group["end_date"].max()]
alumni_filtered_exp_df = exp_df[exp_df["end_date"].notna()]
alumni_filtered_edu_df = edu_df[edu_df["end_date"].notna()]
req_edu_df = alumni_filtered_edu_df.loc[
    alumni_filtered_edu_df["institution"] == INSTITUTION_FILTER
]
req_edu_df_most_recent = req_edu_df.groupby("id").apply(most_recent_row).droplevel(0)

req_exp_df = alumni_filtered_exp_df.loc[
    (alumni_filtered_exp_df["institution"] == INSTITUTION_FILTER)
    | (alumni_filtered_exp_df["group"].isin(GROUP_FILTER))
]
req_exp_df_most_recent = (
    req_exp_df.groupby("id").apply(most_recent_row).droplevel(0)
)

In [59]:
alumni_exp_ids = req_exp_df_most_recent.index.unique()
alumni_edu_ids = req_edu_df_most_recent.index.unique()
alumni_common_ids = list(set(alumni_exp_ids).union(alumni_edu_ids))
alumni_member_df = info_json_df.loc[alumni_common_ids]
for m_key, m_value in alumni_member_df.iterrows():
    if m_key in req_edu_df_most_recent.index:
        if (
            req_edu_df_most_recent.loc[m_key, "degree"] == "Bachelors"
            and req_edu_df_most_recent.loc[m_key, "institution"] == INSTITUTION_FILTER
        ):
            current_academic_role = "Undergraduate Student"
        if req_edu_df_most_recent.loc[m_key, "degree"] in ["PhD", "Masters"]:
            current_academic_role = "Graduate Student"
    elif m_key in req_exp_df_most_recent.index:
        current_academic_role = req_exp_df_most_recent.loc[m_key, "role"]
    else:
        current_academic_role = ""
    alumni_member_df.loc[m_key, "academic_role"] = str(current_academic_role)
alumni_member_df['academic_role'] = alumni_member_df['academic_role'].fillna("")

In [60]:
create_page(
    "alumni_members.html.j2",
    "alumni_members.html",
    general=general,
    alumni_members=alumni_member_df.to_dict("index"),
)

##### Individual People Page

In [75]:
document_df = read_member_data_jsons("documents.json")
outreach_df = (
    read_member_data_jsons("outreach.json")
    .groupby("id")
    .apply(lambda x: x.to_dict(orient="records"))
    .reset_index(name="info")
    .set_index("id")
)

projects_df_grouped = (
    projects_df.groupby("id")
    .apply(lambda x: x.to_dict(orient="records"))
    .reset_index(name="info")
    .set_index("id")
)
awards_df_grouped = (
    awards_df.groupby("id")
    .apply(lambda x: x.to_dict(orient="records"))
    .reset_index(name="info")
    .set_index("id")
)

for member_id, member_data in info_json_df.iterrows():
    if member_id in current_member_df.index:
        info_json_df.at[member_id, "academic_role"] = current_member_df.loc[
            member_id, "academic_role"
        ]
        info_json_df.at[member_id, "current_project_title"] = current_member_df.loc[
            member_id, "current_project_title"
        ]
    elif member_id in alumni_member_df.index:
        info_json_df.at[member_id, "academic_role"] = alumni_member_df.loc[member_id, "academic_role"]

for person_id, person_data in info_json_df.iterrows():
    create_page(
        "individual_person.html.j2",
        f"members/{person_id}/{person_id}.html",
        general=general,
        member_id=person_id,
        member_data=person_data,
        socials=social_links_df.to_dict("index"),
        documents=document_df.to_dict("index"),
        education=edu_df_grouped.to_dict(orient="index"),
        experience=exp_df_grouped.to_dict(orient="index"),
        projects=projects_df_grouped.to_dict(orient="index"),
        awards=awards_df_grouped.to_dict("index"),
        outreach=outreach_df.to_dict("index"),
        section_headings=INDIVIDUAL_MEMBER_SECTION_MAP,
        content=article_content_df.to_dict("index"),
    )